### Import and Load Packages 

In [ ]:
# Load packages
import pandas as pd
import numpy as np
from datetime import date
import csv
import re
import datetime
import os
from sklearn.feature_selection import VarianceThreshold
from pandas import read_csv
from sklearn.datasets import load_breast_cancer
from datetime import date
import kaplanmeier as km
from sklearn.decomposition import PCA
import pylab as pl
from itertools import cycle
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
%matplotlib inline
from qqman import qqman
from pandas_plink import read_plink
from scipy.stats import ttest_ind
import math
import os
from lifelines import KaplanMeierFitter
from pandas import DataFrame
import pandas as pd
import seaborn as sns
sns.set(color_codes=True)
import numpy as np
from pandas.plotting import scatter_matrix
from sklearn import preprocessing
from numpy import set_printoptions
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import datasets
import seaborn as sns; sns.set()
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
#from xgboost import XGBClassifier
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import ElasticNet
from numpy import arange
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import pandas as pd
from google.cloud import bigquery

import os
import matplotlib.pyplot as plt
import textwrap
import re
from tqdm import tqdm
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import metrics
from pprint import pprint

### 2. Set up BigQuery Connection

Set up connection

In [ ]:
user_id = 'kbechler'
nero_gcp_project = 'som-nero-nigam-starr'
cdm_project_id = 'som-nero-nigam-starr'
cdm_dataset_id = 'starr_omop_cdm5_deid_20211213'
work_project_id = 'som-nero-nigam-starr'
work_dataset_id = f'{user_id}_explore'
cdm_subset_dataset_id = 'cdm_subset'
cohort_table_id = 'exp_2022'
cohort_id = 21

In [ ]:
# Use correct path whether you are local or Nero
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/kbechler/adc.json' 

# Set correct Nero project
os.environ['GCLOUD_PROJECT'] = nero_gcp_project
client = bigquery.Client(project = work_project_id)

In [ ]:
# Import list of patients for held out test set
heldout_test_set = pd.read_csv('/home/kbechler/test_set.csv')

In [ ]:
test_patients = tuple(heldout_test_set.patient_id)

### Create training and test set in BigQuery 

In [ ]:
sql = "select * from som-nero-nigam-starr.kbechler_explore.exp_2022"
exp_2022 = client.query(sql).to_dataframe()

In [ ]:
exp_2022.head()

In [ ]:
df_clmbr = pd.read_csv('/home/kbechler/piton_private/df_clmbr_2022.csv')

In [ ]:
df_clmbr.head()

In [ ]:
len(df_clmbr)

In [ ]:
print(df_clmbr.patient_id.nunique())

In [ ]:
len(df_clmbr[df_clmbr.label == True])

In [ ]:
test = df_clmbr[df_clmbr.patient_id.isin(test_patients)]

In [ ]:
len(test)

In [ ]:
test.head()

In [ ]:
test.label.unique()

In [ ]:
print(sum(test.label == True))

In [ ]:
query = """

CREATE OR REPLACE TABLE
`som-nero-nigam-starr.kbechler_explore.exp_test` AS
SELECT *
FROM 
`som-nero-nigam-starr.kbechler_explore.exp_2022` exp_2022
WHERE
subject_id IN{}""".format(test_patients)

client.query(query).result();

In [ ]:
sql = "select * from som-nero-nigam-starr.kbechler_explore.exp_test"
exp_test = client.query(sql).to_dataframe()

In [ ]:
query = """

CREATE OR REPLACE TABLE
`som-nero-nigam-starr.kbechler_explore.exp_train` AS
SELECT *
FROM 
`som-nero-nigam-starr.kbechler_explore.exp_2022` exp_2022
WHERE
subject_id NOT IN{}""".format(test_patients)

client.query(query).result();

In [ ]:
sql = "select * from som-nero-nigam-starr.kbechler_explore.exp_train"
exp_train = client.query(sql).to_dataframe()

In [ ]:
print(len(exp_test))
print(len(exp_train))

In [ ]:
query = """

CREATE OR REPLACE TABLE
`som-nero-nigam-starr.kbechler_explore.out_test` AS
SELECT *
FROM
`som-nero-nigam-starr.kbechler_explore.out_2022` out_2022
WHERE
subject_id IN{}""".format(test_patients)

client.query(query).result();

In [ ]:
query = """

CREATE OR REPLACE TABLE
`som-nero-nigam-starr.kbechler_explore.out_train` AS
SELECT *
FROM 
`som-nero-nigam-starr.kbechler_explore.out_2022` out_2022
WHERE
subject_id NOT IN{}""".format(test_patients)

client.query(query).result();